# Rapport: Implémentation de Hadoop "from scratch" en Java

<a id=index />


Index:
====================
* [Etape 1:](#etape-1) faire un programme séquentiel non parallélisé qui compte le nombre d'occurrences des mots dans un fichier. 

     - [1.1](#1-1) Premier comptage en séquentiel pur
         + [`count`](#count)
         + [`fileToArrayList`](#file-to-array-list)
         + [`separeWordsInLine`](#separe-words-in-line)
     - [1.2](#1-2) Premier tri en séquentiel pur
         + [`sortByFrequency`](#sort-by-frequency)
     - [1.3](#1-3) Deuxième tri alphabétique en séquentiel pur
     - [1.4](#1-4) Test du programme séquentiel sur le code forestier de Mayotte
     - [1.5](#1-5) Les 50 mots du code de la déontologie de la police nationale
     - [1.6](#1-6) Les 50 mots du code du domaine public fluvial
     - [1.7](#1-7) Les 50 mots du code de la santé publique
     - [1.8](#1-8) Chronométrage du programme séquentiel
     - [TODO 1.9](#1-9) Travailler sur des plus gros fichiers


* [Etape 2:](#etape-2) travailler avec plusieurs ordinateurs en réseau

    - [TODO 2.1](#2-1) Nom court, nom long
    - [2.2](#2-2) Adresse ip
    - [TODO 2.3](#2-3) Du nom vers l’IP
    - [TODO 2.4](#2-4) De l’IP vers le nom
    - [TODO 2.5](#2-5) Ping pong à l’intérieur!
    - [TODO 2.6](#2-6) Ping pong à l’extérieur
    - [2.7](#2-7) Calculer en ligne de commande sur l’ordinateur local
    - [2.8](#2-8) Calculer en ligne de commande sur un ordinateur distant
    - [TODO 2.9](#2-9) Calculer à distance sans mot de passe
    
    
* [Etape 3:](#etape-3) travailler avec des fichiers locaux ou sur un serveur NFS

    - [3.1](#3-1) Chemin absolu
    - [3.2](#3-2) Un fichier dans le répertoire personnel
    - [3.3](#3-3) Ou se trouve le fichier dans le répertoire personnel
    - [3.4](#3-4) Un dossier et un fichier dans le répertoire temporaire
    - [3.5](#3-5) Trois ordinateurs A B C. On commence avec A. Utilisation du serveur NFS
    - [3.6](#3-6) Trois ordinateurs A B C. On continue sur B et sur C. Utilisation du serveur NFS
    - [3.7](#3-7) Trois ordinateurs A B C. On commence avec A. Utilisation des disques locaux
    - [3.8](#3-8) Trois ordinateurs A B C. On continue sur B et sur C. Utilisation des disques locaux
    - [3.9](#3-9) Depuis A, copier de A vers B avec les disques locaux
    - [3.10](#3-10) Depuis A, copier de B vers C avec les disques locaux
    

* [Etape 4:](#etape-4) lancer des programmes java à distance manuellement
    
    - [4.1](#4-1) Exportation en JAR
    - [4.2](#4-2) Exécution sur disque dur local
    - [4.3](#4-3) Copie du JAR et exécution distante


* [Etape 5:](#etape-5) lancer des programmes en ligne de commande depuis java et afficher la sortie standard et la sortie d’erreur

    - [5.1](#5-1) Un programme MASTER java qui lance un autre programme en ligne de commande!
    - [5.2](#5-2) Un programme MASTER java qui gère les erreurs de lancement d’un autre programme en ligne de commande.
    - [5.3](#5-3) Un programme MASTER java qui lance un slave.jar en ligne de commande.
    
    
* [Etape 6:](#etape-6) gérer les timeout du MASTER

     - [6.1](#6-1) Un SLAVE qui simule un calcul de 10 secondes
     - [6.2](#6-2) Gérer les timeout au niveau du MASTER
     
     
* [Etape 7:](#etape-7) déployer automatiquement le programme SLAVE sur un ensemble de machines

     - [7.1](#7-1) Un programme DEPLOY : Test de connection SSH multiple
     - [7.2](#7-2) Un programme DEPLOY : copie de slave.jar multiple
     
     
* [Etape 8:](#etape-8) nettoyer un ensemble de machines avec CLEAN

    - [8.1](#8-1) Un programme “CLEAN” qui nettoie les machines distantes
    - [8.2](#8-2) Vérification du DEPLOY et du CLEAN
    
    
* [Etape 9:](#etape-9) lancer le programme SLAVE sur un ordinateur à distance

    - [9.1](#9-1) CLEAN et DEPLOY
    - [9.2](#9-2) Master lançant Slave à distance
    
    
* [Etape 10:](#etape-10) MapReduce - SPLIT et MAP

    - [10.1](#10-1) Un MASTER qui déploie les splits
    - [10.2](#10-2) Un SLAVE qui fait la phase de map
    
    
* [Etape 11:](#etape-11) MapReduce - SHUFFLE


<a id=etape-1 />

## Etape 1: faire un programme séquentiel non parallélisé qui compte le nombre d'occurrences des mots dans un fichier.

[Retour index](#index)

<a id=1-1 />


## 1.1 Premier comptage en séquentiel pur

>Implémentez un logiciel en java qui compte le nombre d’occurrences des mots d’un fichier d’entrée de manière non parallélisée (monothread, une seul thread), en utilisant un seul processeur. <br>
Quelle structure de donnée est la plus pertinente pour stocker les résultats: List, HashMap ou HashSet ou une autre ? Pour quelle raison ? <br>
Testez votre programme avec un fichier d’entrée input.txt avec comme contenu:

Il faut stocker à la fois le mot et son compte donc l'utilisation d'une `List` n'est pas pertinente. `HashSet` est plus rapide que `HashMap` mais elle ne propose pas de fonction de tri. Comme pour les questions suivantes nous cherchons à trier notre dictionnaire, nous utiliserons une `HashMap`

In [3]:
import java.io.IOException;
import java.nio.file.Files;
import java.nio.file.Paths;
import java.text.ParseException;
import java.util.*;


### `Count` 

Méthode comptant les mot d'un fichier d'input: 

>Input: [String] Le nom du fichier <br>
>Output: [Map] Un Map avec en clé un mot et en valeur son nombre d'occurence.
    

In [4]:
public Map<String, Integer> count(String fileName){
    List<String> lines = fileToArrayList(fileName);
    List<String> words = separeWordsInLine(lines);

    Map<String, Integer> mapWordCount = new HashMap<>();
    for (String word :words) {
        if(mapWordCount.containsKey(word.toLowerCase())){
            mapWordCount.put(word.toLowerCase(), mapWordCount.get(word.toLowerCase()) + 1);
        }else{
            mapWordCount.put(word.toLowerCase(), 1);
        }
    }
    return mapWordCount;
}

<a id=file-to-array-list /></a>
#### `fileoArrayList`

Méthode convertissant un fichier en liste de ligne

>Input: [String] Le nom du fichier <br>
>Output: [List<String>] liste de ligne du fichier


In [5]:
public List<String> fileToArrayList(String fileName){
        List<String> lines = new ArrayList<>();
        try{
            lines = Files.readAllLines(Paths.get(fileName));
        }catch (IOException e){
            System.out.println(e.getMessage());
        }
        return lines;
    }

<a id=separe-words-in-line /></a>
#### `separeWordsInLine`

Méthode convertissant un list de ligne en Input en une list de mot composant les lignes. 
La méthode ne prends pas en compte les lignes vides.
    
>Input: [List<String>] List contenant les ligne <br>
>Output: [List<String>] List contenant les mots

In [6]:
public List<String> separeWordsInLine(List<String> lineArray){
    List<String> wordArray = new ArrayList<>();
    for (String line:lineArray) {
        String[] tempWordInLine = line.split(" "); 
        for (String word : tempWordInLine) {
            if(!word.isEmpty())//Utilisé dans les questions suivantes pour enlever les lignes vides
                wordArray.add(word);
        }
    }
    return wordArray;
}

On utilise notre fonction `count` pour créer notre dictionnaire de word count.

In [7]:
//Comment to make JupyterLab run
//public static void  main(String[] args){
        long startTime = System.currentTimeMillis();
        Map<String, Integer> mapCount = count("data/input.txt");
        System.out.println(mapCount);
//    }

{deer=2, car=3, river=4, beer=2}


<a id=1-2 />


## 1.2 Premier tri en séquentiel pur

>Modifiez votre programme pour trier par nombre d'occurrences:

On crée une nouvelle fonction `sortByFrequency` qui se chargera de trier notre dictionaire et 
`display` qui affichera nos résultats.

<a id=sort-by-frequency /></a>
## `sortByFrequency`

In [8]:
private Map<String, Integer> sortByFrequency(Map<String, Integer> mapToSort) {
    //Creation of an ArrayList of Entries to sort the content of the dictionnary
    List<Map.Entry<String, Integer>> keySorted = new ArrayList<>(mapToSort.entrySet());
    keySorted.sort(new Comparator<Map.Entry<String, Integer>>() {
        //Override of regular sort method to suitour needs
        @Override
        public int compare(Map.Entry<String, Integer> s, Map.Entry<String, Integer> t1) {
            return t1.getValue().compareTo(s.getValue());
        }
    });
    //Creation of sorted dictionary with LinkedHashMap to keep the sorted order
    Map<String, Integer> mapSorted = new LinkedHashMap<>();
    for (Map.Entry<String, Integer> entry: keySorted) {
        mapSorted.put(entry.getKey(), entry.getValue());
    }
    return mapSorted;
}

<a id=display /></a>

### `display`

In [9]:
private void display(Map<String, Integer> mapSorted) {
    System.out.println("Affichage par Quantite");
    int i =0;
    for (Map.Entry<String, Integer> entry: mapSorted.entrySet()){
        if(i>50)
            break;
        System.out.println(entry.getKey()+ " : " +  entry.getValue());
        i++;
    }
}

In [10]:
//Main
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
river : 4
car : 3
deer : 2
beer : 2


<a id=1-3 />

## 1.3 Deuxième tri alphabétique en séquentiel pur

>Modifiez le programme pour trier alphabétiquement pour les mots à égalité du nombre d’occurrences:


On modifie notre méthode `sortByFrequency` pour trier notre dictionnaire de manière alphabétique et séquentiel pur. On a:

In [11]:
private Map<String, Integer> sortByFrequency(Map<String, Integer> mapToSort) {
    //Creation of an ArrayList of Entries to sort the content of the dictionnary
    List<Map.Entry<String, Integer>> keySorted = new ArrayList<>(mapToSort.entrySet());
    keySorted.sort(new Comparator<Map.Entry<String, Integer>>() {
        //Override of regular sort method to suitour needs
        @Override
        public int compare(Map.Entry<String, Integer> s, Map.Entry<String, Integer> t1) {
            //Ajout du trie alphabétique pour les mots ayant le même nombre d'occurence
            if(t1.getValue() == s.getValue()){
                return s.getKey().compareTo(t1.getKey());
            }
            return t1.getValue().compareTo(s.getValue());
        }
    });
    //Creation of sorted dictionary with LinkedHashMap to keep the sorted order
    Map<String, Integer> mapSorted = new LinkedHashMap<>();
    for (Map.Entry<String, Integer> entry: keySorted) {
        mapSorted.put(entry.getKey(), entry.getValue());
    }
    return mapSorted;
}

In [12]:
//Main
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
river : 4
car : 3
beer : 2
deer : 2


<a id=1-4 />

## 1.4 Test du programme séquentiel sur le code forestier de Mayotte

>Testez ensuite votre programme avec le code forestier de Mayotte disponible sur github forestier_mayotte.txt :<br> https://github.com/legifrance/Les-codes-en-vigueur <br>
Votre programme a-t-il fonctionné du premier coup ? <br>
Vérifiez en ouvrant le fichier texte qu’il contient bien du texte et non du code HTML. <br>
Ne perdez pas de temps à corriger les éventuelles erreurs dues aux caractères spéciaux ou à des mots suspects ou illisibles (de toutes façons par la suite il y aura du chinois dans le texte).

In [13]:
//Main
long startTime = System.currentTimeMillis();
Map<String, Integer> mapCount = count("data/forestier_mayotte.txt");
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
de : 12
biens : 8
ou : 8
code : 6
forestier : 6
des : 5
: : 4
aux : 4
communes : 4
dispositions : 4
forestiers : 4
gestion : 4
le : 4
les : 4
mayotte : 4
tous : 4
à : 4
législative : 3
partie : 3
accordé : 2
agroforestiers : 2
agroforestiers. : 2
agroforestière : 2
aides : 2
article : 2
attachées : 2
bonne : 2
bénéfice : 2
constitue : 2
dont : 2
démembrer : 2
elle : 2
est : 2
et : 2
fait : 2
forestière : 2
garanties : 2
ii : 2
l'engagement : 2
l'unité : 2
l021 : 2
leur : 2
livre : 2
ne : 2
par : 2
partie. : 2
pas : 2
prioritairement : 2
propriétaires : 2
propriété : 2
préliminaire : 2


<a id=1-5 />

## 1.5 Les 50 mots du code de la déontologie de la police nationale

>Testez votre programme avec le code de déontologie de la police nationale disponible sur github deontologie_police_nationale.txt : https://github.com/legifrance/Les-codes-en-vigueur <br>
De même ne perdez pas de temps à filtrer les caractères spéciaux ou autres mots bizarres. Pourquoi ? Car nous travaillerons ensuite sur des textes en chinois, japonais, arabe et d’autres langues. Si vous implémentez une étape de filtrage ici en français elle ne servira à rien par la suite. Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ?

Les 5 premiers mots ressemblant à des mots sont:
* police : 35
* article : 20
* nationale : 15
* titre : 12
* fonctionnaires : 10

In [14]:
//Main
long startTime = System.currentTimeMillis();
Map<String, Integer> mapCount = count("data/deontologie_police_national.txt");
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
de : 98
la : 50
police : 35
des : 33
et : 33
le : 27
à : 25
les : 24
article : 20
↬ : 19
nationale : 15
en : 13
est : 13
titre : 12
ou : 11
qui : 11
fonctionnaires : 10
: : 9
aux : 9
code : 9
fonctionnaire : 9
l'autorité : 9
par : 9
du : 8
leur : 8
ses : 8
au : 7
devoirs : 7
déontologie : 7
il : 7
ne : 7
a : 6
dans : 6
cas : 5
commandement. : 5
faire : 5
l'ordre : 5
ordres : 5
pour : 5
sa : 5
se : 5
sont : 5
tout : 5
doit : 4
droits : 4
elle : 4
nationale. : 4
pas : 4
responsabilité : 4
si : 4
subordonné : 4


<a id=1-6 />

## 1.6 Les 50 mots du code du domaine public fluvial 

>Testez votre programme avec le code du domaine public fluvial domaine_public_fluvial.txt.
Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ? 

Les 5 premiers mots ressemblant à des mots sont:
* article : 107
* bateau : 74
* tribunal : 72
* lieu : 60
* bureau : 50

In [15]:
//Main
long startTime = System.currentTimeMillis();
Map<String, Integer> mapCount = count("data/domaine_public_fluvial.txt");
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
de : 630
le : 425
la : 364
du : 347
et : 266
les : 232
des : 222
à : 209
est : 173
dans : 150
par : 124
sur : 123
ou : 118
en : 109
article : 107
↬ : 103
au : 97
un : 79
bateau : 74
pour : 73
tribunal : 72
l'article : 68
aux : 66
il : 66
lieu : 60
; : 56
qui : 54
: : 50
bureau : 50
que : 46
d'un : 44
navigation : 43
où : 43
son : 43
code : 42
titre : 42
bateaux : 41
se : 41
d'immatriculation : 40
domicile : 38
a : 37
juge : 36
délai : 34
propriétaire : 34
s'il : 33
créanciers : 32
être : 31
doit : 30
saisie : 30
et, : 29
certificat : 28


<a id=1-7 />

## 1.7 Les 50 mots du code de la santé publique

>Testez votre programme avec le code de la santé publique sante_publique.txt.
Quels sont les 5 premiers mots (qui ressemblent à des mots) parmi les 50 premiers de la liste triée résultat ? 

Les 5 premiers mots ressemblant à des mots sont:
* article : 23070
* santé : 14473
* directeur : 8446
* conditions : 7864
* l'agence : 7750

In [16]:
//Main
long startTime = System.currentTimeMillis();
Map<String, Integer> mapCount = count("data/sante_publique.txt");
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
display(mapSorted);

Affichage par Quantite
de : 190781
la : 82333
des : 67761
à : 65500
les : 62322
et : 60980
le : 56644
du : 48500
ou : 39584
en : 31700
par : 30694
au : 25694
dans : 25076
article : 23070
l'article : 22994
↬ : 21810
est : 21700
un : 20480
; : 19388
l. : 18410
pour : 16860
: : 16166
aux : 16136
sont : 14946
santé : 14473
une : 13842
sur : 12622
r. : 11224
que : 11038
qui : 9744
d'un : 9664
directeur : 8446
être : 8046
conditions : 7864
peut : 7842
l'agence : 7750
ne : 7708
d'une : 7372
conseil : 7306
son : 7294
cas : 6932
il : 6662
général : 6632
ce : 6222
ces : 6186
a : 6038
dispositions : 5836
leur : 5762
1° : 5648
pas : 5602
2° : 5470


<a id=1-8 />

## 1.8 Chronométrage du programme séquentiel

>Chronométrer votre programme sur le code de la santé publique.


In [17]:
//Main
//Timer du count
long startTime = System.currentTimeMillis();
Map<String, Integer> mapCount = count("input.txt");
long endTime = System.currentTimeMillis();
long totalTimeCount = endTime - startTime;
//Timer du sort
startTime = System.currentTimeMillis();
Map<String, Integer> mapSorted = sortByFrequency(mapCount);
endTime = System.currentTimeMillis();
long totalTimeSort = endTime - startTime;
//Display
System.out.println("Counting time is:" + totalTimeCount + " ms");
System.out.println("Sorting time is:" + totalTimeSort + " ms");

input.txt
Counting time is:49 ms
Sorting time is:87 ms


<a id=1-9 />

## 1.9 Travailler sur des plus gros fichiers

>Testez votre programme sur un cas réel: un extrait de toutes les pages internet transformées au format texte brut (format WET). Toutes les pages sur  internet au format texte sont disponibles sur http://commoncrawl.org/the-data/get-started/ : chaque mois, environ 3 milliards de pages web, soit 250 To de données sont stockées. Ces données sont disponibles par tranche de moins d’1Go environ, vous travaillerez sur une tranche de 380Mo. <br>
J’ai choisi une tranche en particulier pour avoir une comparaison entre nous (vous pouvez tester sur d’autres tranches si vous voulez). Téléchargez cette tranche ici: <br>
https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2017-13/segments/1490218189495.77/wet/CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet.gz 
Décompressez et obtenez le fichier CC-MAIN-20170322212949-00140-ip-10-233-31-227.ec2.internal.warc.wet  <br>
Il s’agit d’une tranche contenant un ensemble de sites internet au format texte brut (WET).
Testez votre programme avec ce fichier en entrée. Chronométrez-le. <br>


<a id=etape-2 />

# Etape 2: travailler avec plusieurs ordinateurs en réseau.

[Retour index](#index)

<a id=2-1 />

## 2.1 Nom court, nom long

>Quel est le nom COURT de votre ordinateur (le nom simple sans le domaine) ?   quel est le nom LONG de votre ordinateur (le nom avec le domaine) ? Comment les connaître en ligne de commande ? Sur les ordinateurs de l’école, est-il possible d’obtenir ces noms autrement qu’en ligne de commande ? 

Pour obtenir le nom court de notre ordinateur on tape la ligne de commande:
```shell
vincent@vincent-ThinkPad-T540p:~$ hostname
```
Pour obtenir le nom long la ligne de commande est:
<pre><code>
   TODO
</code></pre>



<a id=2-2 />

## 2.2 Adresse ip

>Comment connaître les adresses (plusieurs) IP de votre ordinateur en ligne de commande ? Autrement (en passant par un site internet par exemple) ? 

Pour connaitre les adresses IP de notre ordinateur on peut taper en ligne de commande:
    
```shell

vincent@vincent-ThinkPad-T540p:~$ ifconfig
 
```

<a id=2-3 />

## 2.3 Du nom vers l’IP

>Comment à partir du nom d’un ordinateur, obtenir les adresses IP en ligne de commande ? 

Obtenir à partir du nom d'un ordinateur les adresse IP :

```shell
 vincent@vincent-ThinkPad-T540p:~$ TODO
```

<a id=2-4 />

## 2.4 De l’IP vers le nom

>Comment, à partir d’une adresse IP, obtenir les noms associés en ligne de commande ? 

Obtenir à partir de l'adresse IP, les noms associés:

```shell
vincent@vincent-ThinkPad-T540p:~$ TODO
```

<a id=2-5 />

## 2.5 Ping pong à l’intérieur!

>Testez la communication avec d’autres ordinateurs (pas le vôtre) depuis le réseau de l’école en utilisant la commande ping (pour arrêter le ping faire CTRL + C). suivi du nom court, du nom long, de l’IP. Les trois méthodes fonctionnent-elles ?

Pour tester la comunication avec d'autres ordinateurs on peut faire les commandes:
```shell
vincent@vincent-ThinkPad-T540p:~$ ping c123-01
    TODO
```

<a id=2-6 />

## 2.6 Ping pong à l’extérieur

On peut utiliser les mêmes commandes que précédemment mais en étant connecter en VPN au réseau de l'école.
```shell
vincent@vincent-ThinkPad-T540p:~$ TODO 
```

<a id=2-7 />

## 2.7 Calculer en ligne de commande sur l’ordinateur local

>Comment lancer un calcul en ligne de commande sur votre ordinateur (par exemple 2 + 3) ? Parmi les multiples réponses possibles, lesquelles permettent de lancer le calcul et d’obtenir le résultat en appuyant une seule fois sur la touche <Entrée> ?

Pour calculer en ligne de commande on peut taper:

```bash
vincent@vincent-ThinkPad-T540p:~$ calc 2+3
```

<a id=2-8 />

## 2.8 Calculer en ligne de commande sur un ordinateur distant

>Comment lancer un calcul  (par exemple 2 + 3) en ligne de commande sur un autre ordinateur (à distance) ? Il faudra certainement vous authentifier avec un mot de passe. Comment obtenir le résultat du calcul immédiatement après avoir tapé son mot de passe ?

Pour ce connecter à distance on utilise la commande `ssh` pour calculer à distance on utilise la commande:
```shell
vincent@vincent-ThinkPad-T540p:~$ ssh vrichard@c123-01 calc 2+3
```

<a id=2-9 />

## 2.9 Calculer à distance sans mot de passe

>Comment lancer un calcul à distance en utilisant SSH sans taper le mot de passe et en une seule ligne de commande (c’est à dire qu’on appuie sur <Entrée> et on a le résultat directement)?

# TODO
Pour se calculer à distance sans mot de passe il faut paramétrer une clé RSA pour être reconnue par les ordinateurs de l'école sans avoir à taper notre mot de passe.
On peut ensuite taper la commande précédente.


<a id=etape-3 />

# Etape 3: travailler avec des fichiers locaux ou sur un serveur NFS.

[Retour index](#index)

<a id=3-1 />

## 3.1 Chemin absolu

>Quel est le chemin absolu de votre répertoire personnel, votre home directory ? (commandes “cd” puis ”pwd”)


Le chemin absolu une fois connecté en `ssh` au ordinateur de l'école est:
```shell
vrichard@c130-05 ~ $ pwd
/cal/homes/vrichard
```

<a id=3-2 />

## 3.2 Un fichier dans le répertoire personnel

>Créez un fichier fperso.txt contenant le texte “bonjour” dans votre répertoire personnel (sur un ordinateur de l’école). 
Vérifiez le contenu du fichier avec cette commande exactement:
cat ~/fperso.txt

```shell
vrichard@c130-05 ~ $ cat ~/fperso.txt
bonjour
```

<a id=3-3 />

## 3.3 Ou se trouve le fichier dans le répertoire personnel

>Ce fichier est-il sur le disque dur de l’ordinateur ou autre part ? Comment savoir où est stocké physiquement ce fichier, à l’aide de quelle commande ? 

Ce fichier est stocker sur le cloud de mon espace personnel on peut le vérifier avec la commande:
```shell
vrichard@c130-05 ~ $ df -k fperso.txt
Sys. de fichiers                        blocs de 1K Utilisé Disponible Uti% Monté sur
zfs-b232.enst.fr:/export/homes/vrichard     3145728   49152    3096576   2% /cal/homes/vrichard
```

Le `Sys. de fichier` nous indique que notre fichier est situé sur le volume `zfs-b232.enst.fr:/export/homes/vrichard` donc il n'est pas sur le système.

<a id=3-4 />

## 3.4 Un dossier et un fichier dans le répertoire temporaire

>Créez un dossier /tmp/<votre nom d’utilisateur> en remplaçant <votre nom d’utilisateur>.<br>
Créez un fichier ftemp.txt dans le répertoire /tmp/<votre nom d’utilisateur> .<br>
Vérifiez le contenu du fichier avec cette commande exactement:<br>
cat /tmp/<votre nom d’utilisateur>/ftemp.txt <br>
Ce dossier et ce fichier sont-ils sur le disque dur de l’ordinateur ou autre part ? Comment savoir où sont stockés physiquement ces éléments, à l’aide de quelle commande ?


On crée un dossier /tmp/vrichard:
```shell
vrichard@c130-05 ~ $ mkdir /tmp/vrichard
```
puis on crée le fichier et on le modifie

```shell
vrichard@c130-05 ~ $ touch /tmp/vrichard/ftemp.txt
```

Modification du fichier:

```shell
vrichard@c130-05 ~ $ nano /tmp/vrichard/ftemp.txt
```

```shell
vrichard@c130-05 ~ $ cat /tmp/vrichard/ftemp.txt
bonjour
```

On observe où le fichier est situé:
```shell
vrichard@c130-05 ~ $ df -k /tmp/vrichard/ftemp.txt
Sys. de fichiers      blocs de 1K Utilisé Disponible Uti% Monté sur
/dev/mapper/vg-lv_tmp     9545920   42164    8999132   1% /tmp
```

Ici on voit que le système de fichier est `/dev/mapper/vg-lv_tmp` donc il est bien enregistrer sur cette machine

<a id=3-5 />

## 3.5 Trois ordinateurs A B C. On commence avec A. Utilisation du serveur NFS

>Pour les questions suivantes, utilisez trois ordinateurs: A, B C. <br>
Connectez vous physiquement (avec un clavier, une souris et un écran) sur l’ordinateur A. <br>
Sur A, créez un fichier text.txt contenant le texte “mon texte sur NFS” dans votre répertoire personnel. <br>
Vérifiez que le fichier existe et vérifiez son contenu. Pour cela, sur A, utilisez la commande : <br>
cat ~/text.txt

On créé notre fichier en ajoutant le texte "mon texte sur NFS":

```shell
vrichard@c130-05 ~ % echo "mon texte sur NFS" > text.txt
```

On vérifie avec la commande cat:

```shell
vrichard@c130-05 ~ % cat ~/text.txt 
mon texte sur NFS
```
    

<a id=3-6 />

## 3.6 Trois ordinateurs A B C. On continue sur B et sur C. Utilisation du serveur NFS

>Connectez-vous à B (physiquement ou à distance) et vérifiez que le fichier text.txt est également présent dans votre répertoire personnel. Pour cela, sur B, utilisez la commande :<br>
cat ~/text.txt <br>
De même, connectez-vous à C et vérifiez que text.txt est aussi présent. <br>
Remarquez que vous n’avez pas copié le fichier mais qu’il est présent sur A, B et C grâce au serveur NFS.


Pour vérifier que le fichier existe bien on effectue la ligne de commande par ssh sur les deux machines:

```shell
vrichard@c130-05 ~ % ssh vrichard@c130-07 "cat ~/text.txt"
mon texte sur NFS
vrichard@c130-05 ~ % ssh vrichard@c130-08 "cat ~/text.txt"
mon texte sur NFS
```

<a id=3-7 />

## 3.7 Trois ordinateurs A B C. On commence avec A. Utilisation des disques locaux

>Déconnectez vous de B et de C et revenez sur l’ordinateur A. <br>
Sur A, créez un dossier /tmp/<votre nom d’utilisateur> et un fichier local.txt contenant le texte “mon texte sur disque local” dans ce dossier /tmp/<votre nom d’utilisateur>. <br>
Vérifiez que le fichier existe et vérifiez son contenu. Pour cela, sur A, utilisez la commande :
cat /tmp/<votre nom d’utilisateur>/local.txt


On réutilise les mêmes commandes que précédemment avec le nom de fichier `local.txt`

```shell
vrichard@c130-05 ~ % echo "mon texte sur disque local" > /tmp/vrichard/local.txt
vrichard@c130-05 ~ % cat tmp/vrichard/local.txt
mon texte sur disque local
```

<a id=3-8 />

## 3.8 Trois ordinateurs A B C. On continue sur B et sur C. Utilisation des disques locaux

>Connectez-vous à B et C (physiquement ou à distance) et vérifiez que le dossier <votre nom d’utilisateur> ainsi que le fichier local.txt ne sont pas présent dans /tmp . Pour cela vérifiez avec la commande: <br>
ls /tmp


Comme précédemment on effectue ces commandes par ssh:
```shell
vrichard@c130-05 ~ % ssh vrichard@c130-07 "ls /tmp"
```
De même pour la machine c130-08 on peut voir que les dossier ne sont pas présent.

<a id=3-9 />

## 3.9 Depuis A, copier de A vers B avec les disques locaux

>Comment, à partir de A, transférer le fichier /tmp/local.txt sur B (dans /tmp/<votre nom d’utilisateur>/local.txt) en utilisant scp ? Vérifiez que le fichier est bien présent sur B. Attention: si vous avez une erreur “no such file or directory” (ou l’équivalent français), vous devez d’abord créer le répertoire /tmp/<votre nom d’utilisateur>/ avec la commande mkdir -p associée à un ssh pour l’ordinateur distant.


On commence par créer le dossier avec la commande:
```shell
vrichard@c130-05 ~ % ssh vrichard@c130-07 "mkdir -p /tmp/vrichard"
```
Puis on copie le fichier avec la commande:
```shell
vrichard@c130-05 ~ % scp /tmp/vrichard/local.txt c130-07:/tmp/vrichard/
```

<a id=3-10 />

## 3.10 Depuis A, copier de B vers C avec les disques locaux

>Comment, à partir de A, transférer le fichier de B (depuis /tmp/<votre nom  d’utilisateur>/local.txt) vers C (dans /tmp/<votre nom d’utilisateur>/local.txt) ? <br>Vérifiez que le fichier est bien présent sur C. De même que la question précédentes, vous devez créer les répertoires /tmp/<votre nom d’utilisateur>/ correspondants.

On commence par créer le dossier avec la commande:
```shell
vrichard@c130-05 ~ % ssh vrichard@c130-08 "mkdir -p /tmp/vrichard"
```
Puis on copie le fichier avec la commande:
```shell
vrichard@c130-05 ~ % scp c130-07:/tmp/vrichard/local.txt c130-08:/tmp/vrichard/
```

<a id=etape-4 />

# Etape 4: lancer des programmes java à distance manuellement

[Retour index](#index)

>Un premier programme SLAVE sous Eclipse <br>
Faire un programme Java nommé “SLAVE” qui calcule 3+5, affiche le résultat, sous Eclipse (Pour lancer Eclipse: Menu applications>développement), lancer ce programme dans Eclipse (flèche verte “exécuter”)


<a id=4-1 />

## 4.1 Exportation en JAR

>Exporter le programme Java en slave.jar exécutable (Java ARchive dite Runnable) sous Eclipse. Attention de bien vérifier que le JAR est de type “Runnable”/”exécutable”.


In [18]:
public class Slave {
    public static void main (String[] args){
        int a = 3+5;
        System.out.println(a);
    }
}

<a id=4-2 />

## 4.2 Exécution sur disque dur local

>Créez le répertoire /tmp/<votre nom d’utilisateur>/ <br>
Copiez  slave.jar exécutable dans le répertoire /tmp/<votre nom d’utilisateur>/ <br>
Testez et Lancer le slave.jar en ligne de commande sur votre ordinateur local.

Pour executer le .jar on applique la commande:
```shell
vrichard@c130-05 ~ % java -jar tmp/vrichar/SLAVE.jar
8
```

<a id=4-3 />

## 4.3 Copie du JAR et exécution distante

>Depuis la machine A contenant /tmp/<votre nom d’utilisateur>/slave.jar  <br>
Créez à distance sur la machine B (s’il n’existe pas) un répertoire /tmp/<votre nom d’utilisateur>/ <br>
Copiez slave.jar sur la machine B dans le répertoire /tmp/<votre nom d’utilisateur>/ <br>
Exécutez à distance (depuis A sur la machine B) le slave.jar. <br>
Quelle est la commande tapée pour effectuer cette dernière action ?


On copie le fichier:
```shell
vrichard@c130-05 ~ % scp /tmp/vrichard/SLAVE.jar c130-07:/tmp/vrichard/
```
Puis on l'execute:
```shell
vrichard@c130-05 ~ % ssh c130-07 "java -jar /tmp/vrichard/SLAVE.jar"
8
```


<a id=etape-5 />

# Etape 5: lancer des programmes en ligne de commande depuis java et afficher la sortie standard et la sortie d’erreur

[Retour index](#index)

<a id=5-1 />

## 5.1 Un programme MASTER java qui lance un autre programme en ligne de commande!

>Ecrire un programme java nommé “MASTER” qui lance la commande suivante en utilisant ProcessBuilder: <br>
ls -al /tmp <br>
(vous pouvez aussi tester cette commande dans un terminal avant)
Récupérer et afficher la sortie de cette commande. <br>
Vous devez utiliser ProcessBuilder de cette façon: <br>
ProcessBuilder pb = new ProcessBuilder("ls", “-al”, “/tmp”);


On utilise la class `ProcessBuilder` pour lancer notre programme extérieur.  <br>
La méthode `printInputStream` nous permet de récupérer et afficher un élément d'un reader. <br>
Ici on créé un nouveau `InputStreamReader` encapsulant la sortie de notre `ProcessBuilder` et nous permettant de l'afficher.

In [9]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.Reader;
import java.util.stream.Collectors;

public class Master {
    public Master() {
    }

    public static void main(String[] args) throws IOException {
        ProcessBuilder pb = new ProcessBuilder("ls", "-al", "/tmp");
        Process p = pb.start(); 
        //Récupération de l'InpuStream (des éléments de sorties) de cette commande
        printInputReader(new InputStreamReader(p.getInputStream()));
    }

    public static void printInputReader(Reader reader) {
        BufferedReader readerOutput = new BufferedReader(reader);
        //Récupération et impression de tous les sortie renvoyer par la ligne de commande
        System.out.print((String)readerOutput.lines().collect(Collectors.joining(System.lineSeparator())));
    }
}

<a id=5-2 />

## 5.2 Un programme MASTER java qui gère les erreurs de lancement d’un autre programme en ligne de commande 

>Modifiez votre programme”MASTER” pour qu’il affiche la sortie d’erreur en cas d’erreur lors de l’exécution de la commande. Testez la sortie d’erreur avec une commande qui échoue avec un sortie d’erreur. Essayez par exemple d’exécuter la commande “ls /jesuisunhero”.

On reprends notre programme précédant en appelant la méthode `printInputReader` pour chaque sortie qui nous intéresse. donc ici une pour la sortie standard `InputStream` et une pour la sortie d'erreur `ErrorStream`.

In [8]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.Reader;
import java.util.stream.Collectors;

public class Master {
    public Master() {
    }

    public static void main(String[] args) throws IOException {
        ProcessBuilder pb = new ProcessBuilder("ls", "/jesuisunhero");
        Process p = pb.start();
        //Récupère et affiche la sortie standard du process p
        printInputReader(new InputStreamReader(p.getInputStream()));
        //Récupère et afficher la sortie d'erreur du process p
        printInputReader(new InputStreamReader(p.getErrorStream()));
    }

    public static void printInputReader(Reader reader) {
        BufferedReader readerOutput = new BufferedReader(reader);
        System.out.print((String)readerOutput.lines().collect(Collectors.joining(System.lineSeparator())));
    }
}

<a id=5-3 />

## 5.3 Un programme MASTER java qui lance un slave.jar en ligne de commande

>Modifiez votre programme “MASTER” pour qu’il lance “SLAVE”, c’est à dire slave.jar situé sur la même machine que “MASTER” dans le dossier /tmp/<votre nom d’utilisateur>/slave.jar 

On modifie la commande effectué par le `ProcessBuilder` pour qu'il execute le SLAVE.jar. en unité de commande ce la donne: 

```shell
java -jar tmp/vrichar/SLAVE.jar
```

On change donc les arguments du constructeur en conséquence

In [10]:
import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStreamReader;
import java.io.Reader;
import java.util.stream.Collectors;

public class Master {
    public Master() {
    }

    public static void main(String[] args) throws IOException {
        //Nouveau ProcessBuilder executant un jar
        ProcessBuilder pb = new ProcessBuilder("java", "-jar", "/tmp/vrichard/SLAVE.jar");
        Process p = pb.start();
        //Récupère et affiche la sortie standard du process p
        printInputReader(new InputStreamReader(p.getInputStream()));
        //Récupère et afficher la sortie d'erreur du process p
        printInputReader(new InputStreamReader(p.getErrorStream()));
    }

    public static void printInputReader(Reader reader) {
        BufferedReader readerOutput = new BufferedReader(reader);
        System.out.print((String)readerOutput.lines().collect(Collectors.joining(System.lineSeparator())));
    }
}

<a id=etape-6 />

# Etape 6: gérer les timeout du MASTER

[Retour index](#index)

<a id=6-1 />

## 6.1 Un SLAVE qui simule un calcul de 10 secondes

>Modifiez votre programme SLAVE pour qu’il simule une attente de 10 secondes avant d’afficher le résultat du calcul 3+5. Pour cela utilisez <br>
`Thread.sleep(10000);` <br>
Vérifiez le bon fonctionnement du SLAVE et constatez qu’il y a 10 secondes entre le démarrage du SLAVE et l’affichage du résultat. Attention de ne rien afficher avant les 10 secondes pour que la question suivante fonctionne correctement. <br>
Générez de nouveau le slave.jar. Copiez-le en écrasant le slave.jar dans le dossier /tmp/<votre nom d’utilisateur>/slave.jar <br>
Testez le lancement à partir de MASTER.


L'element try/catch est nécessaire par définition de la méthode `Thread.sleep`

In [11]:
public class Slave {
    public static void main (String[] args){
        try {
            //Attente de 10 secondes
            Thread.sleep(10000);
        } catch (InterruptedException e) {
            e.printStackTrace();
        }
        int a = 3+5;
        System.out.println(a);
    }
}

<a id=6-2 />

## 6.2 Gérer les timeout au niveau du MASTER

>Modifier le MASTER pour qu’il attende que quelque chose soit écrit dans la sortie standard (sans erreur) ou dans la sortie d’erreurs du SLAVE pendant un certain temps maximum. Au bout du temps imparti le MASTER considère un timeout. <br>
Il arrête les éventuels threads (si vous utilisez des threads - non obligatoire) s’occupant des sorties et/ou stoppe le processus créé avec ProcessBuilder. <br>
Vous avez la solution ci-dessous pour implémenter les TESTs suivants: <br>
TEST1 : Testez le bon fonctionnement du timeout en lançant le SLAVE avec un timeout de 2 secondes sur les sorties (standard et d’erreur). Le timeout étant plus court (au niveau du MASTER 2 secondes) que le temps de calcul du SLAVE (10 secondes), le MASTER doit arrêter les éventuels threads (si vous en utilisez) et le processus. <br>
TEST 2: Testez ensuite avec un timeout de 15 secondes, il ne devrait pas y avoir de timeout. <br>
TEST 3: Testez ensuite en changeant le SLAVE pour qu’il écrive non plus dans la sortie standard (sans erreur) mais dans la sortie d’erreur. Pour cela, remplacez dans le Slave les System.out.print… par System.err.print… <br>


<a id=etape-7 />

# Etape 7: déployer automatiquement le programme SLAVE sur un ensemble de machines

[Retour index](#index)

<a id=7-1 />

## 7.1 Un programme DEPLOY : Test de connection SSH multiple

<a id=7-2 />

## 7.2 Un programme DEPLOY : copie de slave.jar multiple

<a id=etape-8 />

# Etape 8: nettoyer un ensemble de machines avec CLEAN

[Retour index](#index)

<a id=8-1 />

## 8.1 Un programme “CLEAN” qui nettoie les machines distantes

<a id=8-2 />

## 8.2 Vérification du DEPLOY et du CLEAN

<a id=etape-9 />

# Etape 9: lancer le programme SLAVE sur un ordinateur à distance.

[Retour index](#index)

<a id=9-1 />

## 9.1 CLEAN et DEPLOY

<a id=9-2 />

## 9.2 Master lançant Slave à distance.

<a id=etape-10 />

# Etape 10: MapReduce - SPLIT et MAP

[Retour index](#index)

<a id=10-1 />

## 10.1 Un MASTER qui déploie les splits

<a id=10-2 />

## 10.2 Un SLAVE qui fait la phase de map

<a id=etape-11 />

# Etape 11: MapReduce - SHUFFLE

[Retour index](#index)